In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import os
import re
%matplotlib inline

from rapidfuzz import fuzz, process, utils
import jellyfish
import snowflake.connector

import time
# from time import time

import pandasql

pd.options.display.max_colwidth = 10000
pd.set_option('display.max_rows', 100000)


In [2]:
cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_S', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')


In [3]:
main_start = time.time()

In [4]:
q = """SELECT * FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE I
WHERE  INVOICE_UPDATE_DATETIME BETWEEN '2021-08-10 01:00:00.000' AND '2021-08-10 02:00:00.000' 
"""

In [5]:
ENTITY = pd.read_sql(q,cnx)

In [6]:
q2 = """  SELECT distinct INVOICE_UPC_CASE as PRODUCT_UPC_CASE ,GTIN,
PRODUCT_DESCRIPTION from FDC_STAGE.DASCI.RK_PROD_CAT_MAPPED_TEMP"""

In [7]:
RK_PROD_CAT_MAPPED = pd.read_sql(q2,cnx)

In [8]:
match_start_time = time.time()

In [9]:
data = pandasql.sqldf("SELECT DISTINCT INVOICE_UPDATE_DATETIME,INVOICE_UPC_CASE,INVOICE_UPC_PACK,INVOICE_PACKAGE_DESCRIPTION , GTIN, PRODUCT_DESCRIPTION FROM \
                      ENTITY,RK_PROD_CAT_MAPPED WHERE INVOICE_UPC_CASE = PRODUCT_UPC_CASE OR INVOICE_UPC_PACK = PRODUCT_UPC_CASE ")

In [10]:
match_end_time = time.time()
print("matching_time," , (match_end_time -match_start_time ))

matching_time, 1233.934531211853


In [11]:
len(data)

80276

In [12]:
data['new_invoice_description'] = ''
# data['new_invoice_description1'] = ''

In [13]:
data.columns

Index(['INVOICE_UPDATE_DATETIME', 'INVOICE_UPC_CASE', 'INVOICE_UPC_PACK',
       'INVOICE_PACKAGE_DESCRIPTION', 'GTIN', 'PRODUCT_DESCRIPTION',
       'new_invoice_description'],
      dtype='object')

In [14]:
len(data)

80276

In [15]:
def calculate_fuzz_ratio(x,y):
    return fuzz.partial_ratio(x,y)

In [16]:
def calculate_soundex_ratio(x,y):
    Xsoundex = jellyfish.soundex(x)
    Ysoundex = jellyfish.soundex(y)
    return calculate_fuzz_ratio(Xsoundex,Ysoundex)

In [17]:
def mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold):
    flag = "NEED TO INSPECT"
#Fuzzy call
    fuzzy_ratio = calculate_fuzz_ratio(invoice_description ,product_description)

    if fuzzy_ratio >= fuzzy_threshold:
        
        return product_description
    else:
#Soundex Call
        soundex_fuzz_ratio = calculate_soundex_ratio(invoice_description ,product_description)
        if soundex_fuzz_ratio >= soundex_threshold:
            return product_description 
        else:
            return flag


In [18]:
def left(string, length):
    return string[:length]

def right(string, length):
    return string[-length:]

In [19]:
GTIN_LENGTH =  [14,13,12,11,10]
def mapping_function(data,i, match_upc, fuzzy_threshold, soundex_threshold):
    flag = "NEED TO INSPECT"
    invoice_description     = data['INVOICE_PACKAGE_DESCRIPTION'][i]
    
    product_description     = data['PRODUCT_DESCRIPTION'][i]
#    invoice_product_is_id   = data['PRODUCT_IS_ID'][0]
#    product_master_key      = data['PRODUCT_MASTER_CATALOG_KEY'][0]
    GTIN                    = data['GTIN'][i]
    
    
#Exact match  
    
    for N in GTIN_LENGTH:
        if invoice_description == product_description:
            #invoice_product_is_id = product_master_key
            print("WOWW EXACT MATCH")
        elif len(GTIN) == 14:                                   #Length 14 
                if invoice_description != product_description:    # if description doesn't not match
                    return mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold)        #Call mapping logic
        elif right(GTIN,N) == match_upc and invoice_description != product_description:
            
            return mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold) 
        elif left(GTIN,N) == match_upc and invoice_description != product_description:
            
            return mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold) 
        else:
            return flag



            

            
        


In [20]:
data.columns

Index(['INVOICE_UPDATE_DATETIME', 'INVOICE_UPC_CASE', 'INVOICE_UPC_PACK',
       'INVOICE_PACKAGE_DESCRIPTION', 'GTIN', 'PRODUCT_DESCRIPTION',
       'new_invoice_description'],
      dtype='object')

In [21]:
type(data['GTIN'][0])

str

#### Time taken for 82524 records to process: 77.631

In [22]:
start = time.time()
for i in range(0,len(data)):
    data['new_invoice_description'][i] = mapping_function(data,i,data['INVOICE_UPC_CASE'][i],50,70)

for i in range(0,len(data)):
    for n in GTIN_LENGTH:
        if  data['new_invoice_description'][i] == "NEED TO INSPECT":
            
            data['new_invoice_description'][i] = mapping_function(data,i,data['INVOICE_UPC_PACK'][i],50,70)
end = time.time()


WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MATCH
WOWW EXACT MAT

In [23]:
print("algotihm time", (end-start))

algotihm time 84.26208758354187


In [24]:
mian_end = time.time()

print("full_process_time" ,(mian_end -main_start ))


full_process_time 1353.382281780243


In [25]:


need_to_inspect = data[data['new_invoice_description'] == 'NEED TO INSPECT']


In [26]:
need_to_inspect.to_csv('/home/rkothari/Downloads/need_to_inspect.csv')

In [27]:
data.to_csv('/home/rkothari/Downloads/mapped_new_mapped_descriptions.csv')

In [28]:
# for i in range(0,5000):
#     live_query = f'''SELECT DISTINCT GTIN,GTIN_12_UPC,PRODUCT_DESCRIPTION,INVOICE_UPC_CASE,INVOICE_UPC_PACK,INVOICE_PACKAGE_DESCRIPTION FROM RK_LEN_CASE_PACK,PRODUCT_MASTER_CATALOG
#                 WHERE INVOICE_UPC_CASE = GTIN OR INVOICE_UPC_PACK = GTIN  OR INVOICE_UPC_CASE = GTIN_12_UPC OR INVOICE_UPC_PACK= GTIN_12_UPC limit 100'''

#     try:
#         con.execute(live_query)
#     
#     except:
#         print(i.GS1_COMPANY_PREFIX, i.VALID_UPC, i.PRODUCT_DESCRIPTION)
# con.close()